In [1]:
import paho.mqtt.client as mqtt
import numpy as np
import io

def recv_markers(client, userdata, msg):
    data = np.load(io.BytesIO(msg.payload))
    print(data['tvecs'], data['rvecs'])

client = mqtt.Client()
client.on_message = recv_markers
client.connect('172.31.1.150', 1883)
client.subscribe('putzini/markers')
client.loop_forever()

[[ -97.213165 -376.77942   408.6455  ]
 [        nan         nan         nan]
 [        nan         nan         nan]
 [        nan         nan         nan]
 [        nan         nan         nan]
 [        nan         nan         nan]
 [        nan         nan         nan]
 [        nan         nan         nan]
 [        nan         nan         nan]
 [        nan         nan         nan]
 [-102.604675 -303.4879     55.606667]
 [        nan         nan         nan]
 [        nan         nan         nan]
 [        nan         nan         nan]
 [        nan         nan         nan]
 [        nan         nan         nan]
 [        nan         nan         nan]
 [        nan         nan         nan]
 [        nan         nan         nan]
 [        nan         nan         nan]
 [        nan         nan         nan]
 [        nan         nan         nan]
 [        nan         nan         nan]
 [        nan         nan         nan]
 [        nan         nan         nan]
 [        nan         nan

KeyboardInterrupt: 